In [ ]:
# 导包
import pandas as pd
import os
from pyecharts.charts import Bar3D
# 改变相对路径
os.chdir(r'D:\PycharmProjects\pythonStudy\Pandas')
print(os.getcwd())

In [ ]:
sheet_names = ['2015','2016','2017','2018','会员等级']
sheet_dict = pd.read_excel('./data/sales.xlsx',sheet_name=sheet_names)
sheet_dict

In [ ]:
# 3.查看上诉数据类型
print(type(sheet_dict))

In [ ]:

# 4.从字典中获取2015表的数据
sheet_dict['2015']

# 5.查看2015表的详细数据
sheet_dict['2015'].info
# 5.查看2015表的统计数据
sheet_dict['2015'].describe()

In [ ]:
# 8.查看个表的详细信息，统计信息
for i in sheet_names[:-1]:
    print(sheet_dict[i].info())


## 2.数据预处理

In [ ]:
# 1.处理缺失值
for i in sheet_names[:-1]:
    # print(sheet_dict[i].info)
    # 2.2 处理缺失值
    sheet_dict[i]=sheet_dict[i].dropna()
    # 2.3 过滤出合法值，即金额1元一下的
    sheet_dict[i]=sheet_dict[i][sheet_dict[i]['订单金额']>1]
    # 2.3新增1列
    sheet_dict[i]['max_year_date'] = sheet_dict[i]['提交日期'].max()

In [ ]:
for i in sheet_names[:-1]:
    print(sheet_dict[i].info())

In [ ]:
# 2.5 查看处理后的数据
for i in sheet_names:
    print(sheet_dict[i].info())
    # print(sheet_dict[i].describe())

In [ ]:
sheet_dict.values()

In [ ]:
data_merge=pd.concat(list(sheet_dict.values())[:-1])

In [ ]:
# 3.合并数据集 获取到处理后的DataFrame对象  去除会员等级   [:-1]  0到-1行 
data_merge=pd.concat(list(sheet_dict.values())[:-1])
data_merge

In [ ]:
# 3.合并数据集 获取到处理后的DataFrame对象  去除会员等级   [:-1]  0到-1行 
data_merge=pd.concat(list(sheet_dict.values())[:-1],ignore_index=True)
data_merge


In [ ]:
data_merge.index

In [ ]:
for i in data_merge:
    print(i)

In [ ]:
data_merge['max_year_date']

In [ ]:
# 3.2 给表新增一列 年
data_merge['year']=data_merge['max_year_date'].dt.year
# 3.3 给表新增一列 data_interval
data_merge['data_interval']=data_merge['max_year_date']-data_merge['提交日期']
# 3.4 把上诉时间间隔，转成对应的天数（数值）
data_merge['data_interval']=data_merge['data_interval'].dt.days
# 3.5 查看最终合并结果
data_merge

In [ ]:

# 3.2 给表新增一列 年
data_merge['year'] = data_merge['max_year_date'].dt.year
# 3.3 给表新增一列 data_interval
data_merge['date_interval'] = data_merge['max_year_date'] - data_merge['提交日期']
# 3.4 把上诉时间间隔，转成对应的天数（数值）
data_merge['date_interval'] = data_merge['date_interval'].dt.days
# 3.5 查看最终合并结果
data_merge

## 3.数据分析

In [ ]:
# 1. 按照 year, 会员ID分组, 分别计算: r(最近购买时间), f(频次), m(金额)三项的 原始数据.
rfm_gb=data_merge.groupby(['year','会员ID'],as_index=False).agg({
    'date_interval':'min',
    '订单号':'count',
    '订单金额':'sum'
})

# 2. 修改列名.
# rfm_gb.columns = ['year', '会员ID', 'r', 'f', 'm']
rfm_gb

In [ ]:
# 1. 按照 year, 会员ID分组, 分别计算: r(最近购买时间), f(频次), m(金额)三项的 原始数据.
rfm_gb = data_merge.groupby(['year', '会员ID'], as_index=False).agg({
    'date_interval': 'min',         # 间隔时间的最小值 -> r(recency)
    '订单号': 'count',               # 购买频次的数量 -> f(frequency)
    '订单金额': 'sum'                # 购买金额的和 -> m(monetary)
})

# 2. 修改列名.
rfm_gb.columns = ['year', '会员ID', 'r', 'f', 'm']
rfm_gb

In [ ]:
rfm_gb.iloc[:2,2:]
rfm_gb

In [ ]:
rfm_gb.iloc[:,2:].describe()

In [ ]:
# 3. 因为采用的是三分法，所以需要确定4个值，才能划分三个区间
# 3.1 查看r,f,m的分数位
rfm_gb.iloc[:,2:].describe().T

In [ ]:
# 3.2 自定义区间
r_bins = [-1,79,255,365]
f_bins = [0,2,5,130]
m_bins = [0,69,1199,206252]

In [ ]:
# 4. 具体的划分区间的动作, 获取 r, f, m三项的 具体评分. 
# 场景1: 演示 自动划分区间, 即: 给定区间数, 由Pandas自动划分区间.

In [ ]:
# 4. 具体的划分区间的动作, 获取 r, f, m三项的 具体评分. 
# 场景1: 演示 自动划分区间, 即: 给定区间数, 由Pandas自动划分区间.
pd.cut(rfm_gb['r'], bins=3).unique()        # Categories类型: [(-0.365, 121.667], (121.667, 243.333], (243.333, 365.0]]

# 场景2: 演示 自定义区间, 即: 我们给定数字, 由Pandas进行具体的划分.
pd.cut(rfm_gb['r'], bins=r_bins).unique()   # Categories类型: [(-1, 79], (79, 255], (255, 365]]

# 场景3: 自定义区间 + 评分.
pd.cut(rfm_gb['r'], bins=r_bins, labels=[3, 2, 1])      # 可以, 但是不推荐, 因为值写死了. 
pd.cut(rfm_gb['f'], bins=f_bins, labels=[1, 2, 3])      # 可以, 但是不推荐, 因为值写死了. 
pd.cut(rfm_gb['m'], bins=m_bins, labels=[1, 2, 3])      # 可以, 但是不推荐, 因为值写死了. 

# 场景4: 最终版, 自定义区间 + 评分(结合区间值 列表的长度获取)
# r(Recency): 最后一次购买的时间, 越小越好, 评分是: 3, 2, 1
rfm_gb['r_label'] = pd.cut(rfm_gb['r'], bins=r_bins, labels=[i for i in range(len(r_bins) - 1, 0, -1)]) # range(3, 0, -1) -> [3, 2, 1]    
# f(Frequency): 购买频次, 越大越好, 评分是: 1, 2, 3
rfm_gb['f_label'] = pd.cut(rfm_gb['f'], bins=f_bins, labels=[i for i in range(1, len(f_bins))]) # range(1, 4) -> [1, 2, 3]    
# m(Monetary): 购买金额, 越大越好, 评分是: 1, 2, 3
rfm_gb['m_label'] = pd.cut(rfm_gb['m'], bins=m_bins, labels=[i for i in range(1, len(m_bins))]) # range(1, 4) -> [1, 2, 3]    
rfm_gb